In [2]:
#import sys
#sys.path = ['', '/Users/panda/miniconda2/lib/python27.zip',
#            '/Users/panda/miniconda2/lib/python2.7', '/Users/panda/miniconda2/lib/python2.7/plat-darwin', 
#            '/Users/panda/miniconda2/lib/python2.7/plat-mac', 
#            '/Users/panda/miniconda2/lib/python2.7/plat-mac/lib-scriptpackages', 
#            '/Users/panda/miniconda2/lib/python2.7/lib-tk', '/Users/panda/miniconda2/lib/python2.7/lib-old', 
#            '/Users/panda/miniconda2/lib/python2.7/lib-dynload', '/Users/panda/miniconda2/lib/python2.7/site-packages', 
#            '/Users/panda/miniconda2/lib/python2.7/site-packages/setuptools-19.6.2-py2.7.egg']

import seaborn as sns
import json
import pandas as pd
import numpy as np
from pandas.io import data
from pandas import Series, DataFrame
import nltk
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.linear_model as linear_model
import sklearn.svm as svm
import sklearn.cross_validation as cross_validation
import sklearn.cluster as cluster
from sklearn.ensemble import AdaBoostClassifier
import sklearn.datasets as sk_data
from sklearn.cluster import KMeans
from sklearn import mixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import sklearn.utils as utils

import statsmodels.api as sm

from patsy import dmatrices

import seaborn as sns

import numpy as np


from scipy.spatial.distance import pdist, squareform
    
import gmplot
import collections


C:\Users\himani\Anaconda2\lib\site-packages\pandas\io\data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


## Class Station
This is the class to aggregate data. Latter we will use its location information to draw our results on map

In [3]:
'''
Cluster is based on each station's information, so this class is the abstract of station.

'''
class Station(object):
    
    # @station_id int
    # Use an integer number to initilize an station instance
    def __init__(self, station_id):
        self.station_id = station_id
        self.features = ['TEMP', 'DEWP', 'SLP', 'STP',
                         'VISIB','WDSP','MXSPD',
                         'GUST','MAX','MIN','SNDP','FOG',
                        'RAIN','SNOW','HAIL','THUNDER','TORNADO']
        
        self.missingValue = [9999.9, 9999.9, 9999.9, 9999.9,
                            999.9, 999.9, 999.9,999.9,
                             9999.9, 9999.9,999.9,
                            -1,-1,-1,-1,-1,-1]
        
        self.META = {}
        self.data = {}
        self.location = [0.0,0.0]
        
        for x in xrange(1,13,1):
            self.META[x] = {}
            self.data[x] = {}
            for name in self.features:
            
                self.META[x][name] = 0
                self.data[x][name] = 0
    
        
    
    # @month int
    # @data dict, must include all the fields that list in self.features
    # This method will filt all the default or missing value
    def inputData(self, month, data):
        
        for i in xrange(len(self.features)):
            name = self.features[i]
            
            rs = data.get(name, None)
            
            if rs != None and rs != self.missingValue[i] :
                self.data[month][name] += rs
                self.META[month][name] += 1
    
    # @month int
    # return the summary for each month. This is an inner function
    def summaryByMonth(self, month):
        exp = ['FOG','RAIN','SNOW','HAIL','THUNDER','TORNADO']
        result = []
        for kind in self.features:
            
            if kind in exp:
                result.append(self.data[month][kind])
            else:
                v = 0.0
                if self.META[month][kind] != 0:
                    v = (self.data[month][kind]+0.0) / (self.META[month][kind])
                result.append(v)
        return result
                    
    
    # For each station, this method return the summary.
    # This method return the feature vector for analyzing
    def summaryFeatureVector(self):
        rs = []
        for x in xrange(1,13,1):
            rs.extend(self.summaryByMonth(x))
        return rs
    
    # This method return method dictinary
    def summaryByDict(self):
        rs = {}
        for x in xrange(1,13,1):
            rs[x] = self.summaryByMonth(x)
        return rs



### Other helper functions


In [28]:

# Some of the data may contains one bit flag like '*,A,I' and so on
# This method will pop this value if it contains one bit tag
def extractNumber(s):
    last = s[-1]
    l = len(s)
    if ord(last) >= ord('0') and ord(last) <= ord('9'):
        return s
    else:
        return s[0:l-1]

# 6 special weather are represented by 6-bits string in the raw data
# Here I extract them and write the value to its coresponded field  
def extractSpecialWeather(d, s):
    w = ['FOG', 'RAIN', 'SNOW', 'HAIL', 'THUNDER', 'TORNADO']
    if len(s) != 6 :
        print 'alert! not well-formed data'
    
    for i in xrange(6):
        if s[i] == '1':
            d[w[i]] = 1
        else:
            d[w[i]] = 0
    return d
    
# Input the joined data file path
# Read line by line and build a dict with all stations
def buildStationDict(filepath):
    rs = {}
    f = open(filepath)
    names = ['TEMP', 'DEWP', 'SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 
             'GUST', 'MAX', 'MIN', 'SNDP', 'FOG', 'RAIN', 'SNOW', 'HAIL', 
             'THUNDER', 'TORNADO']
    for each in f:
        data = each.strip().split(',')
        station_id = int(data[0])
        month = data[2][4:6]
        month = int(month)
        fdict = {}
        i = 0
        for x in xrange(3, 17, 2):
            fdict[names[i]] = float(data[x])
            i+=1
        fdict['GUST'] = float(data[16])
        
        maxV = extractNumber(data[17])
        minV = extractNumber(data[18])
        
        
        fdict['MAX'] = float(maxV)
        fdict['MIN'] = float(minV)
        fdict['SNDP'] = float(data[20])
        fdict = extractSpecialWeather(fdict, data[21])
        
        if rs.get(station_id) == None:
            p = Station(station_id)
            rs[station_id] = p
        p = rs[station_id]
        p.location = [data[-2],data[-1]]
        p.inputData(month, fdict)
    return rs
        
        

        
# Convert raw data location format
# For the latitude and logitude, E,N will be positive while W,S will be negative
def getLocation(loc):
    rs = [0,0]
    
    la = loc[0]
    lo = loc[1]
    
    end_la = la[-1]
    end_lo = lo[-1]
    
    l1 = len(la)
    l2 = len(lo)
    
    la = la[0:l1-1]
    lo = lo[0: l2-1]
    
    if end_la == 'N':
        rs[0] = float(la) / 100.0
    else:
        rs[0] = float(la) / -100.0
    
    if end_lo == 'E':
        rs[1] = float(lo) / 100.0
    else:
        rs[1] = float(lo) / -100.0
    return rs    
    
# @data pandas.DataFrame  Must contains 3 feilds, 'latitude','longitude','label'
# @filepath str specify where you want to store the result
# @colors, If you have more than 6 labels, you should specify your own color list
def showOnMap(data, filepath,colors = []):
    gmap = gmplot.GoogleMapPlotter(36.161517,-115.164011,16)

    la = list(data['latitude'])
    lo = list(data['longitude'])
    label = list(data['label'])
    
    if len(colors) == 0:
        colors = ['#ff0000','#00ff00','#0000ff','#ffff00','#ff00ff','#00ffff']
    draw_la = collections.defaultdict(list)
    draw_lo = collections.defaultdict(list)

    kind = len(colors)
    for i in range(len(label)):
        try:
            color = colors[int(label[i]) % kind]
        
            draw_la[color].append(la[i])
            draw_lo[color].append(lo[i])
        except:
            print i
            print label[i]

    for each in draw_la.keys():
        gmap.scatter(draw_la[each], draw_lo[each], each, size=10000, marker=False)
    gmap.draw(filepath)   

    
# @data pandas.DataFrame
# @n_clusters  How many clusters you want
# @plot_graph Only to be True if you want to plot the graph, especially when there are only 2 labels
def GMMCluster(data, n_clusters = 3, plot_graph = False):
    colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
    print n_clusters
    gmm = mixture.GMM(n_components=n_clusters, covariance_type='diag')
    gmm.fit(data)
    y_pred = gmm.predict(data)
    
    if plot_graph:   
        plt.scatter(data[:, 0], data[:, 1], s=10,color=colors[y_pred].tolist(), alpha=0.8)
        plt.show()
    return y_pred

'''
data: ndarray. Data matrix
n_clusters: int. How many clusters you want to have.
detail: bool. Whether to out put some detail information
plot_graph: bool. Whether to plot heat graph to compare unsorted and sorted label. 

return type (labels, centers)
'''
def kMeansCluster(data,clusters = 10, detail = False,plot_graph = False):
    colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
    kmeans = KMeans(init='k-means++', n_clusters=clusters, n_init=10)
    kmeans.fit_predict(data)
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_
    error = kmeans.inertia_
    
    if detail:
        print "The total error of the clustering is: ", error
        print '\nCluster labels'
        print labels
        print '\n Cluster Centroids'
        print centroids
        
    if plot_graph:
        
        plt.scatter(data[:, 0], data[:, 1], s=10,color=colors[labels].tolist(), alpha=0.8)
        plt.show()
        #idx = np.argsort(labels)
        #rX = X[idx,:]
        #fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,10))
        #sns.heatmap( rX,xticklabels=False, yticklabels=False, linewidths=0,cbar=True,ax = ax1)
        #sns.heatmap( X,xticklabels=False, yticklabels=False, linewidths=0,cbar=True,ax = ax2)
    return (labels, centroids)

'''
data: ndarry. Data matrix
begin: int. The beginning test number of cluster
end: int. The maximum number of clusters
'''
def estimateCluster(data, end = 11):
    error = np.zeros(end)
    error[0] = 0;
    for k in range(1,end):
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
        kmeans.fit_predict(data)
        error[k] = kmeans.inertia_

    plt.plot(range(1,len(error)),error[1:])
    plt.xlabel('Number of clusters')
    plt.ylabel('Error')


In [29]:
fdict

NameError: name 'fdict' is not defined

### Build the station dictionary

In [5]:
m = buildStationDict('data_with_location.csv')

### Convert station to feature vector

In [6]:
features = []
locations = []    

for station_id in m.keys():    
    station = m[station_id]
    locations.append( getLocation( station.location) )
    features.append(station.summaryFeatureVector())
    

data = np.array(features)


CLUSTERING

In [7]:
ec = estimateCluster(data, 11)
ec

In [8]:
labels,centers = kMeansCluster(data,10)
print labels
print centers

[2 1 0 ..., 0 0 2]
[[  3.57550854e+01   2.68108735e+01   1.01928453e+03 ...,   5.90142672e-02
    6.75421530e-01   2.26977951e-03]
 [  3.49423147e+01   2.64803912e+01  -1.06865627e-11 ...,   3.49544073e-02
    6.28419453e-01   7.59878419e-03]
 [  4.66175805e+01   3.48428485e+01   9.82126305e+02 ...,   1.90288714e-02
    5.67585302e-01   1.96850394e-03]
 ..., 
 [  4.45968033e+01   3.73102191e+01   1.01865005e+03 ...,   7.21153846e-02
    5.48076923e-01   9.61538462e-03]
 [  4.71118729e+01   3.72591446e+01   9.76190305e+02 ...,   1.58620690e-01
    1.45517241e+00  -6.93889390e-18]
 [  1.95578063e+01   1.28512261e+01   8.71576079e+01 ...,   1.72413793e-02
    7.58620690e-01  -2.60208521e-18]]


In [9]:
las = []
los = []
label = []
i = 0
for each in locations:
    las.append(each[0])
    los.append(each[1])
    label.append(labels[i])
    i+=1

rs = {'latitude':las, 'longitude':los, 'label':label}

df = pd.DataFrame(rs)

print df.columns

colors = ['#000000','#0000FF','#8A2BE2',
          '#5F9EA0','#7FFF00','#FF7F50',
          '#DC143C', '#00FFFF','#006400',
         '#8B008B']

    

showOnMap(df, 'cluster.html',colors)

Index([u'label', u'latitude', u'longitude'], dtype='object')


### Do GMM cluster
Do GMM cluster and show it on the map

In [10]:
labels = GMMCluster(data, 15)

15


In [11]:
las = []
los = []
label = []
i = 0
for each in locations:
    las.append(each[0])
    los.append(each[1])
    label.append(labels[i])
    i+=1

rs = {'latitude':las, 'longitude':los, 'label':label}

df = pd.DataFrame(rs)

print df.columns

colors = ['#000000','#0000FF','#8A2BE2',
          '#5F9EA0','#7FFF00','#FF7F50',
          '#DC143C', '#00FFFF','#006400',
         '#8B008B', '#556B2F','#483D8B',
         '#9400D3','#FFD700','#FF4500']

    

showOnMap(df, 'GMM-cluster.html',colors)

Index([u'label', u'latitude', u'longitude'], dtype='object')


## Next part SVM

- First part is the helper functions
- Use SVM to predict special weather
- Draw ROC curve to show our result

In [34]:
import collections

# @weather_str 6-bits string. 
# @in_detail if True, return lables from 0 to 63. If False, return boolean labels
def getSpecialWeatherLabel(weather_str, in_detail = True):
    
    label = 0
    for x in range(6):
        weight = 1 << x
        v = int(weather_str[x])
        label += weight*v
    
    if in_detail:
        return label
    else:
        return label == 0
        

# For each row in the file, get its features and labels
# @filepath input data file path
# @return type: pandas.DataFrame
def extractClassificationTable(filepath):
    f = open(filepath)
    
    fdict = collections.defaultdict(list)
    
    names = ['TEMP', 'DEWP', 'SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 
             'GUST', 'MAX', 'MIN', 'SNDP', 'FOG', 'RAIN', 'SNOW', 'HAIL', 
             'THUNDER', 'TORNADO']
    for each in f:
        data = each.strip().split(',')
        #get month
        month = data[2][4:6]
        month = int(month)
        
        fdict['MONTH'].append(month)
        
        i = 0
        for x in xrange(3, 17, 2):
            fdict[names[i]].append(float(data[x]))
            i+=1
        fdict['GUST'].append(float(data[16]))
        
        maxV = extractNumber(data[17])
        minV = extractNumber(data[18])
        
        
        fdict['MAX'].append(float(maxV))
        fdict['MIN'].append(float(minV))
        fdict['SNDP'].append(float(data[20]))
        fdict['LABEL'].append(getSpecialWeatherLabel(data[21], True))
        
        lati, logi = getLocation([data[-2],data[-1]])
        fdict['LAT'].append(lati)
        fdict['LOG'].append(logi)
    
    frame = pd.DataFrame(fdict)
    
    return frame
        


    
    
    
    

### MULTIPLE REGRESSION

In [35]:
classify_dataframe = extractClassificationTable('data_narrow.csv')
cf = classify_dataframe

In [112]:
names = 'SLP'
names = ['SLP']
sum_values=0
mean=0
count=0
#column_len = len(cf['SLP'])
    #print column_len
for i in range(0,len(names)):
    column_len = cf[names]
    for j in range(0,column_len):
        value =  cf[names][j]
    ##print cf[name][j]
        if (value  != 999.9) or (value  != 9999.9) or (value  != -1):
            mean = mean + value 
            count+=1
    sum_values = (mean / count)
    print sum_values
#return sum_values

3548.1247812


In [138]:
def mean_columns(name,cf):
    sum_values=0
    mean=0
    column_len = len(cf[name])
    count=0
    #print name
    #print name
    for j in range(0,column_len):
        value =  cf[name][j]
        #print value
        if (value  != 999.9) or (value  != 9999.9) or (value  != -1):
            mean = mean + value
            count+=1
    if count!=0:
        sum_values = (mean / count)
    else:
        sum_values=0
    return sum_values

In [139]:
def replace(sum_values,name,cf):
        column_len = len(cf[name])
        for j in range(0,column_len):
            value =  cf[name][j]
            #print value
            if (value  == 999.9) or (value  != 9999.9) or (value  != -1):
                cf[name][j] = sum_values


In [140]:
sum_values=0
names = ['SLP', 'DEWP', 'GUST', 'STP', 'WDSP', 'MXSPD', 
             'TEMP', 'MAX', 'MIN', 'SNDP']
for i in range(0,len(names)):
    print names[i]
    sum_values = mean_columns(names[i],cf)
    print sum_values
    replace(sum_values,names[i],cf)
    if i ==1:
        break


SLP
3418.91302244


C:\Users\himani\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [42]:
import pandas as pd
import statsmodels.api as sm

y = cf['LABEL']
X = cf[['DEWP', 'GUST', 'LAT', 'LOG', 'MAX', 'MIN', 'MXSPD', 'SLP', 'SNDP', 'STP','TEMP', 'WDSP']]
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  LABEL   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     27.49
Date:                Mon, 18 Apr 2016   Prob (F-statistic):           7.17e-63
Time:                        22:42:23   Log-Likelihood:                -67424.
No. Observations:               23787   AIC:                         1.349e+05
Df Residuals:                   23774   BIC:                         1.350e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          2.0192      0.149     13.523      0.000         1.727     2.312
DEWP          -0.0001   1.79e-05     -6.933      0.000        -0.000 -8.89e-05
GUST          -0.0005   6.96e-05     -7.580      0.000        -0.001    -0.000
LAT            0.0027      0.001      2.347      0.019         0.000     0.005
LOG            0.0018      0.000      5.041      0.000         0.001     0.003
MAX           -0.0002      0.000     -1.555      0.120        -0.000  4.36e-05
MIN            0.0002      0.000      1.897      0.058     -6.68e-06     0.000
MXSPD          0.0002      0.000      0.799      0.424        -0.000     0.001
SLP        -1.042e-05   7.41e-06     -1.407      0.159     -2.49e-05   4.1e-06
SNDP          -0.0011      0.000     -8.473      0.000        -0.001    -0.001
STP         2.763e-05   7.21e-06      3.834      0.000      1.35e-05  4.18e-05
TEMP           0.0181      0.001     12.861      0.000         0.015     0.021
WDSP        2.371e-05      0.000      0.065      0.949        -0.001     0.001
==============================================================================
Omnibus:                    16498.347   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           191519.582
Skew:                           3.353   Prob(JB):                         0.00
Kurtosis:                      15.176   Cond. No.                     4.63e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.63e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
import statsmodels.api as sm
#print type(df)
#df = df.convert_objects(convert_numeric=True)
print type(cf)
X = cf[['DEWP', 'GUST', 'MAX', 'MIN', 'MXSPD', 'SLP', 'SNDP', 'STP','TEMP', 'WDSP']].values
print type(X)
print X.shape
#print df.shape
y = cf['LABEL']
print type(y)
#print y
print y.shape
#mod = smf.ols(formula='Average_Score ~ Location + Sleep + Rooms + Service + Value + Clean', data=df)
#res = mod.fit()
#print(res.summary())
#print df
model = sm.OLS(y, X)
results = model.fit()
print results.summary()
print "Confidence Intervals:", results.conf_int()
print "Parameters:", results.params

<class 'pandas.core.frame.DataFrame'>
<type 'numpy.ndarray'>
(23787L, 10L)
<class 'pandas.core.series.Series'>
(23787L,)
                            OLS Regression Results                            
Dep. Variable:                  LABEL   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     420.9
Date:                Mon, 18 Apr 2016   Prob (F-statistic):               0.00
Time:                        22:52:09   Log-Likelihood:                -67578.
No. Observations:               23787   AIC:                         1.352e+05
Df Residuals:                   23777   BIC:                         1.353e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------

Random Forest

from sklearn.ensemble import RandomForestClassifier



y = weather_labels.values
X = classify_dataframe.values

X, y = utils.shuffle(X, y, random_state=1)
print X.shape
print y.shape
print y
train_set_size = 5000
X_train = X[:train_set_size]  # selects first 10000 rows (examples) for train set
y_train = y[:train_set_size]
X_test = X[train_set_size:]   # selects from row 10000 until the last one for test set
y_test = y[train_set_size:]
print(X_train.shape), y_train.shape
print(X_test.shape), y_test.shape




rf = RandomForestClassifier(n_estimators=50,oob_score=True)#number of trees in the forest is 50 , '100' gives just little more accuracy than '50'
rf.fit(X_train,y_train)


pred=rf.predict(X_test)

rf.score(X_test,y_test)#mean accuracy 0f the given test data and labels

from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, pred[:])
roc_auc = auc(fpr, tpr)
print roc_auc  #roc_auc is the vealuation metric for random forest

import sklearn.metrics

print sklearn.metrics.confusion_matrix(y_test,pred)# confusion matrix 
print sklearn.metrics.accuracy_score(y_test, pred)  # 76.5 % areas were classified corrrectly 

import matplotlib.pylab as plt

trees=range(50)
accuracy=np.zeros(50)

for idx in range(len(trees)):
    classifier=RandomForestClassifier(n_estimators=idx + 1)
    classifier=classifier.fit(X_train,y_train)
    predictions=classifier.predict(X_test)
    accuracy[idx]=sklearn.metrics.accuracy_score(y_test, pred)#accuracy score for each of random classifier trees from 1 to 50 is stored in an array
plt.cla()
plt.plot(trees, accuracy)

### Build dataset

In [13]:
import random

f = open('data_with_location.csv')
fin = open('data_narrow.csv','w')

for each in f:
    i = random.random()
    if i < 0.01:
        fin.write(each)

f.close()
fin.close()


In [14]:
classify_dataframe = extractClassificationTable('data_narrow.csv')
weather_labels  =classify_dataframe.pop('LABEL')
print classify_dataframe.shape
print classify_dataframe.head(2)
print classify_dataframe.columns

(23787, 14)
   DEWP   GUST    LAT  LOG   MAX   MIN  MONTH  MXSPD     SLP   SNDP     STP  \
0  10.6  999.9  70.56 -8.4  26.8  12.0      3   27.2  1030.1  999.9  1028.9   
1  20.3  999.9  70.56 -8.4  30.6  23.4      4   23.3  1015.3  999.9  1014.1   

   TEMP  VISIB  WDSP  
0  17.1   19.7  21.9  
1  27.3   24.4  12.9  
Index([u'DEWP', u'GUST', u'LAT', u'LOG', u'MAX', u'MIN', u'MONTH', u'MXSPD',
       u'SLP', u'SNDP', u'STP', u'TEMP', u'VISIB', u'WDSP'],
      dtype='object')


### Split the dataset to train and test part

In [15]:

y = weather_labels.values
X = classify_dataframe.values

X, y = utils.shuffle(X, y, random_state=1)
print X.shape
print y.shape
print y
train_set_size = 5000
X_train = X[:train_set_size]  # selects first 10000 rows (examples) for train set
y_train = y[:train_set_size]
X_test = X[train_set_size:]   # selects from row 10000 until the last one for test set
y_test = y[train_set_size:]
print(X_train.shape), y_train.shape
print(X_test.shape), y_test.shape


(23787L, 14L)
(23787L,)
[False  True  True ...,  True  True  True]
(5000L, 14L) (5000L,)
(18787L, 14L) (18787L,)


### Do SVM classification and show its accuracy

In [16]:
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

svc = svm.SVC(kernel='linear', max_iter=10000)
svc.fit(X_train, y_train)
y_pred_test = svc.predict(X_test)
print "Accuracy of SVM test set:", svc.score(X_test, y_test)


Accuracy of SVM test set: 0.561239154735


C:\Users\himani\Anaconda2\lib\site-packages\scikit_learn-0.17.1-py2.7-win-amd64.egg\sklearn\svm\base.py:224: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


### Plot ROC curve to evaluate our result
When you try to plot ROC curve, make sure your label is binary

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

classifier = OneVsRestClassifier(svc)

y_score = classifier.fit(X_train, y_train).decision_function(X_test)

print y_score.shape


(11894L, 1L)


In [18]:

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test[:], y_score[:])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


##############################################################################
# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[1], tpr[1], label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

RANDOM FOREST 

In [19]:
from sklearn.ensemble import RandomForestClassifier



y = weather_labels.values
X = classify_dataframe.values

X, y = utils.shuffle(X, y, random_state=1)
print X.shape
print y.shape
print y
train_set_size = 5000
X_train = X[:train_set_size]  # selects first 10000 rows (examples) for train set
y_train = y[:train_set_size]
X_test = X[train_set_size:]   # selects from row 10000 until the last one for test set
y_test = y[train_set_size:]
print(X_train.shape), y_train.shape
print(X_test.shape), y_test.shape




(23787L, 14L)
(23787L,)
[False  True  True ...,  True  True  True]
(5000L, 14L) (5000L,)
(18787L, 14L) (18787L,)


In [20]:
rf = RandomForestClassifier(n_estimators=50,oob_score=True)#number of trees in the forest is 50 , '100' gives just little more accuracy than '50'
rf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [21]:
pred=rf.predict(X_test)

In [22]:
rf.score(X_test,y_test)#mean accuracy 0f the given test data and labels

0.76020652578910952

In [23]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, pred[:])
roc_auc = auc(fpr, tpr)
print roc_auc  #roc_auc is the vealuation metric for random forest

0.709112631798


In [24]:
import sklearn.metrics

print sklearn.metrics.confusion_matrix(y_test,pred)# confusion matrix 
print sklearn.metrics.accuracy_score(y_test, pred)  # 76.5 % areas were classified corrrectly 

[[ 3545  3046]
 [ 1459 10737]]
0.760206525789


In [25]:
import matplotlib.pylab as plt

trees=range(50)
accuracy=np.zeros(50)

for idx in range(len(trees)):
    classifier=RandomForestClassifier(n_estimators=idx + 1)
    classifier=classifier.fit(X_train,y_train)
    predictions=classifier.predict(X_test)
    accuracy[idx]=sklearn.metrics.accuracy_score(y_test, pred)#accuracy score for each of random classifier trees from 1 to 50 is stored in an array
plt.cla()
plt.plot(trees, accuracy)